In [1]:
import os
import json
import joblib
import time

import pandas as pd
import random
from IPython.display import display, HTML

In [2]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
#connect to hugging face
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
install
!pip install -q ./transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [4]:
# Installing transformers
!pip install simpletransformers
!pip install transformers
# Importing necessary libraries from simple transformers
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [5]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)

Using:  cuda


In [7]:
headlines_processed = []
clean_news = []
dataset = []
target = []
count = 0

In [8]:
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [9]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_train = details_dict['text_ner']
Y_train = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_valid = details_dict['text_ner']
Y_valid = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text_ner']
Y_test = details_dict['headlines']


In [10]:
print(len(X_train), len(Y_train), len(X_valid), len(Y_valid), len(X_test), len(Y_test))

73280 73280 9160 9160 9160 9160


In [11]:
X_train[1:2]

['Washington   Two Secret Service members resigned Colombia prostitution scandal agency said Tuesday another member forced two cleared serious misconduct . A total 24 people  12 Secret Service members 12 U . Meanwhile top Republican Senate Judiciary Committee said Tuesday wants outside investigation determine whether White House staff members engaged inappropriate behavior Colombia . Chuck Grassley Iowa told CNN . Grassley said wants inspector general Department Homeland Security investigate White House staff members possible involvement controversy embarrassed Secret Service raised questions possible security breach President Barack Obama trip Cartagena Colombia Summit Americans month . Acknowledging inquiry would outside inspector general jurisdiction require Obama invitation Grassley said president keep promise maintain unprecedented openness transparency administration . On Monday White House spokesman Jay Carney said White House Counsel Kathryn Ruemmler conducted internal review w

In [12]:
Y_train[1:2]

['Two Secret Service agents resign Colombia scandal']

In [13]:
details = {
    'headlines' : Y_train,
    'text' : X_train
}

df_train = pd.DataFrame(details)

In [14]:
details = {
    'headlines' : Y_valid,
    'text' : X_valid
}

df_valid = pd.DataFrame(details)

In [15]:
details = {
    'headlines' : Y_test,
    'text' : X_test
}

df_test = pd.DataFrame(details)

In [16]:
# Import


tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', torch_device)


Using:  cuda


In [17]:
# Rename columns as per pretrained model required format
df_train=df_train.rename(columns={'headlines':'target_text','text':'input_text'})
df_valid=df_valid.rename(columns={'headlines':'target_text','text':'input_text'})
df_test=df_test.rename(columns={'headlines':'target_text','text':'input_text'})
#df = {dataset, target}
model_args = Seq2SeqArgs()
# Initializing number of epochs
#model_args.num_train_epochs = 25
model_args.num_train_epochs = 2
# Initializing no_save arg
model_args.no_save = True
model_args.overwrite_output_dir = True

# Initializing evaluation args
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

In [18]:
df_train['target_text'][0]

'U . N .  report highlights unspeakable atrocities North Korea'

In [19]:
df_train['input_text'][0]

'Story highlights United Nations inquiry looking human rights violations North Korea The rights probe documented unspeakable atrocities witness interviews Commission listened prison camp survivors suffered childhoods starvation North Korea rejected testimony slander put forward human scum A mother forced drown baby prison camp inmate compelled eat rodents lizards survive  horrific experiences documented United Nations inquiry human rights violations North Korea . What seen heard far  specificity detail shocking character personal testimony  appears without doubt demand followup action world community accountability behalf Democratic People Republic Korea Michael Kirby chair threemember commission inquiry told U . Human Rights Council Tuesday . Kirby said interim findings based testimony given public hearings Seoul Tokyo last month sources including North Korean defectors former regime officials survivors political prison camps families Japanese South Koreans abducted North Korean agent

In [20]:
df_valid['target_text'][0]

'Blame genetics bad driving study finds'

In [21]:
df_valid['input_text'][0]

' secreted area brain associated activity . The protein helps facilitate communication among brain cells helps retain memory . However people gene variation Cramer studied BDNF secretion limited . These people make errors getgo forget learned time away Cramer said statement . Cramer team researchers set find variant affected driving . They recruited 29 people drive 15 laps simulated course difficult curves turns . Id curious know genetics people get car crashes Cramer said .'

In [22]:
df_test['target_text'][0]

'Soldier slain London street machine gunner drummer dad'

In [23]:
df_test['input_text'][0]

'Story highlights Drummer Lee Riggers Rigby 25 machine gunner became recruiter He real character infantry commanding officer says As drummer stood outside Royal Palaces He also father 2yearold son Jack The British soldier slain gruesome cleaver attack London wellliked infantryman machine gunner served Afghanistan Cyprus became military recruiter ceremonial drummer outside royal palaces military said Thursday . Drummer Lee Rigby 25 part Regimental Recruiting Team London machine gunner part 2nd Battalion Royal Regiment Fusiliers . Rigby 2yearold son Jack UK Ministry Defense said . Jim Taylor commanding officer 2nd Fusiliers said statement . An experienced talented side drummer machine gunner true warrior served distinction Afghanistan Germany Cyprus . Rigby also loved soccer Manchester United soldiers said . He one Battalion great characters always smiling always ready brighten mood fellow Fusiliers Warrant Officer Ned Miller 2nd Fusiliers said statement . Rigby born Crumpsall Manchester

In [24]:
model = Seq2SeqModel(
encoder_decoder_type="bart",
encoder_decoder_name="facebook/bart-large",
args=model_args,
use_cuda=True,
)
# Splitting data into train-test
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.2)


In [25]:
df_train.shape, df_valid.shape, df_test.shape

((73280, 2), (9160, 2), (9160, 2))

In [26]:
len(df_train)//10000

7

In [27]:
def current_time_min():
    time_in_mins = time.time()/60.0
    return round(time_in_mins, 4)

In [28]:
# Training the model and keeping eval dataset as test data
# 16000 for aws, 800 for local VM
batch_size_train = 10000
batch_size_test = batch_size_train//4
number_of_batches = len(df_train)//batch_size_train
count = 0
epocs = model_args.num_train_epochs
acc_time = 0

for i in range(number_of_batches):
    start_time = current_time_min()
    count += 1
    start_offset_train = i * batch_size_train
    start_offset_test = i * batch_size_test
    end_offset_train = start_offset_train + batch_size_train
    end_offset_test = start_offset_test + batch_size_test
    model.train_model(df_train[start_offset_train:end_offset_train], eval_data=df_valid)
    model_filename = "model_ner_" + str(count) + "_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(epocs) + "e.pkl"
    joblib.dump(model, os.path.join(path_models, model_filename))
    results = model.eval_model(df_test)
    batch_time = current_time_min() - start_time
    acc_time = acc_time + batch_time/60
    randint = random.randrange(1, len(df_test), 3)
    results_json = {
        'iteration' : count,
        'start' : start_offset_train,
        'end' : end_offset_train,
        'eval_loss' : results["eval_loss"],
        'batch_time' : batch_time,
        'acc_time' : acc_time
    }
    jsonString = json.dumps(results_json)
    results_filename = "results_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(count)
    results_file = open(os.path.join(path_models, "results.txt"), "a")
    results_file.write(jsonString + '\n')
    results_file.close()
    print("Count: ", count, "Results: ", results)


  0%|          | 0/10000 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1250 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Generating outputs:   0%|          | 0/1145 [00:00<?, ?it/s]

  0%|          | 0/9160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1145 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1145 [00:00<?, ?it/s]

Count:  7 Results:  {'eval_loss': 3.1082379217231115}


In [29]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_train = details_dict['text']
Y_train = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_valid = details_dict['text']
Y_valid = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text']
Y_test = details_dict['headlines']


In [30]:
details = {
    'headlines' : Y_train,
    'text' : X_train
}

df_train = pd.DataFrame(details)

details = {
    'headlines' : Y_valid,
    'text' : X_valid
}

df_valid = pd.DataFrame(details)

details = {
    'headlines' : Y_test,
    'text' : X_test
}

df_test = pd.DataFrame(details)



In [31]:
# Import


tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', torch_device)

# Rename columns as per pretrained model required format
df_train=df_train.rename(columns={'headlines':'target_text','text':'input_text'})
df_valid=df_valid.rename(columns={'headlines':'target_text','text':'input_text'})
df_test=df_test.rename(columns={'headlines':'target_text','text':'input_text'})
#df = {dataset, target}
model_args = Seq2SeqArgs()
# Initializing number of epochs
#model_args.num_train_epochs = 25
model_args.num_train_epochs = 2
# Initializing no_save arg
model_args.no_save = True
model_args.overwrite_output_dir = True

# Initializing evaluation args
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

Using:  cuda


In [32]:
model = Seq2SeqModel(
encoder_decoder_type="bart",
encoder_decoder_name="facebook/bart-large",
args=model_args,
use_cuda=True,
)

In [33]:
# Training the model and keeping eval dataset as test data
# 16000 for aws, 800 for local VM
batch_size_train = 10000
batch_size_test = batch_size_train//4
number_of_batches = len(df_train)//batch_size_train
count = 0
epocs = model_args.num_train_epochs
acc_time = 0

for i in range(number_of_batches):
    start_time = current_time_min()
    count += 1
    start_offset_train = i * batch_size_train
    start_offset_test = i * batch_size_test
    end_offset_train = start_offset_train + batch_size_train
    end_offset_test = start_offset_test + batch_size_test
    model.train_model(df_train[start_offset_train:end_offset_train], eval_data=df_valid)
    model_filename = "model_" + str(count) + "_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(epocs) + "e.pkl"
    joblib.dump(model, os.path.join(path_models, model_filename))
    results = model.eval_model(df_test)
    batch_time = current_time_min() - start_time
    acc_time = acc_time + batch_time/60
    randint = random.randrange(1, len(df_test), 3)
    results_json = {
        'iteration' : count,
        'start' : start_offset_train,
        'end' : end_offset_train,
        'eval_loss' : results["eval_loss"],
        'batch_time' : batch_time,
        'acc_time' : acc_time
    }
    jsonString = json.dumps(results_json)
    results_filename = "results_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(count)
    results_file = open(os.path.join(path_models, "results_2.txt"), "a")
    results_file.write(jsonString + '\n')
    results_file.close()
    print("Count: ", count, "Results: ", results)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/9160 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1145 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1145 [00:00<?, ?it/s]

  0%|          | 0/9160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1145 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1145 [00:00<?, ?it/s]

Count:  7 Results:  {'eval_loss': 3.0440909513740038}
